# Sign Up to Use Google Earth Engine (GEE)

## Developer API

[Google Earth Engine sign up form](https://signup.earthengine.google.com/#!/)

Use of the service is strictly non-commercial. Please consult the T&Cs:

> 1.2 Limitations on Use. Unless otherwise approved by Google in writing, Services may only be used for development, research, or educational purposes; you can use data or figures created by use of Services in research or educational publications authored by you. Services may not be used for sustained commercial purposes unless otherwise approved by Google in writing, but may be evaluated in a production environment. Applications and data created with Services may not be sold or licensed for a fee unless otherwise approved by Google in writing.

[source: Google Earth Engine Terms of Use](https://earthengine.google.com/terms/)

As I'm logged into my Google profile, the form would only accept my gmail address. I selected affiliated with ONS and purpose was to 'investigate land use classification within my locality and nationally'. 

Notification to keep an eye on Gmail inbox over the next week for status update.

I had the confirmation Email come through within the hour, but I have a vague recollection of going through Earth Engine access before.

### Service Account

After reading the documentation and trying a couple of approaches, the method that worked for me was to configure my application as a [service account](https://developers.google.com/earth-engine/guides/service_account), which seems to be the method used for hosting of an application that can authenticate with the Earth Engine API for multiple end users. 

***

## Hello World

1. Ensure you have run `pip install requirements.txt` from the root of this directory. 
2. Ensure you have the required dependencies within `./.git_ignore`. Check README for details on secrets.

In [ ]:
import os

import ee
import toml
from pyprojroot import here

In [ ]:
service_email = toml.load(os.path.join(here(), ".git_ignore", "ee.toml"))["ee"][
    "SERVICE_EMAIL"
]

In [ ]:
credentials = ee.ServiceAccountCredentials(
    service_email,
    os.path.join(here(), ".git_ignore", "ee-lsoa-land-use-02b30d7b328b.json"),
)
ee.Initialize(credentials)

In [ ]:
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

***

## Python client tutorial

Ensure the most recent release of earth engine client is being run with:
`pip install earthengine-api --upgrade`

Link to the [Google Earth Engine API tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)

Improve awareness of data available by browsing the [Earth Engine data catalogue](https://developers.google.com/earth-engine/datasets/catalog)

### GEE-specific terms

The documentation refers to:

**Features**: Geometries. Like Spatial features.

**Images**: Rasters, tiffs and the like.

**Collections**: A resource that combines features & images.

**Scale**: Image resolution.

### GEE Specification

* Coordinates are in longitude latitude order.
* Dates are YYYY-MM-DD, eg '2020-01-01'
* Start date is inclusive, End date is exclusive.


### Tutorial Context

Resources used are:
* MODIS land cover (LC)
* MODIS land surface temp (LST)
* USGS ground elevation (ELV)

**Warning: The LST documentation linked to by the tutorial has a deprecation notice.**

Two are `ee.ImageCollections`. ELV is an image. Things we need from the resource descriptions:
1. Availability
2. Provider
3. GEE snippet
4. Available bands

So for [LC](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1):

![](../../www/modis_lc_metadata.png)

Copy the snippet for use as an endpoint.

In [ ]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection("MODIS/006/MCD12Q1")

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection("MODIS/006/MOD11A1")

# Import the USGS ground elevation image.
elv = ee.Image("USGS/SRTMGL1_003")

### About These Resources

* Resolution, frequency & projection can vary:
    * LST = 1km, daily
    * ELV = 30m, yearly
* GEE will attempt to handle projections & resolution by reprojecting / resampling, to make things consistent. 
* We can manually control the resolution (referred to as scale in GEE).
* We can force no reprojection if needed.
* The resources may come with several bands. Check the resource metadata for details.
* **To visualise, we need to specify the date and band to work with.**

In [ ]:
# Initial date of interest (inclusive).
i_date = "2017-01-01"

# Final date of interest (exclusive).
f_date = "2020-01-01"

# Selection of appropriate bands and dates for LST.
lst = lst.select("LST_Day_1km", "QC_Day").filterDate(i_date, f_date)

Next, we need to define the area of interest. The tutorial states you can upload coordinates or shapefiles and uses the coordinate context of rural and urban Lyons, France.

In [ ]:
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)

At this point, many methods are available for use. The [API developer docs](https://developers.google.com/earth-engine/api_docs) are suggested. This tutorial uses:

>`sample()`: samples the image (does NOT work for an ee.ImageCollection — we'll talk about sampling an ee.ImageCollection later) according to a given geometry and a scale (in meters) of the projection to sample in. It returns an ee.FeatureCollection.  
>`first()`: returns the first entry of the collection.  
>`get()`: to select the appropriate band of your Image/Collection.  
>`getInfo()`: evaluates server-side expression graph and transfers result to client.


The tutorial points out that LST comes with a few important caveats:
* Apparently the description states that a factor of 0.02 should be applied to get units in Kelvin. Though checking the linked desrciption (with a deprecation notice) I could not find this mentioned.
* Then a simple conversion to celcius is used.
* This is a multi-annual mean because of the date range employed. 

In [ ]:
scale = 1000  # scale in meters

# Print the elevation near Lyon, France.
elv_urban_point = elv.sample(u_poi, scale).first().get("elevation").getInfo()
print("Ground elevation at urban point:", elv_urban_point, "m")

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get("LST_Day_1km").getInfo()
print(
    "Average daytime LST at urban point:",
    round(lst_urban_point * 0.02 - 273.15, 2),
    "°C",
)

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get("LC_Type1").getInfo()
print("Land cover value at urban point is:", lc_urban_point)

According to the catalogue metadata, 13 is:
> Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles.

Repeating the same for the rural coordinates:

In [ ]:
# Print the elevation near Lyon, France.
elv_rural_point = elv.sample(r_poi, scale).first().get("elevation").getInfo()
print("Ground elevation at rural point:", elv_rural_point, "m")

# Calculate and print the mean value of the LST collection at the point.
lst_rural_point = lst.mean().sample(r_poi, scale).first().get("LST_Day_1km").getInfo()
print(
    "Average daytime LST at rural point:",
    round(lst_rural_point * 0.02 - 273.15, 2),
    "°C",
)

# Print the land cover type at the point.
lc_rural_point = lc.first().sample(r_poi, scale).first().get("LC_Type1").getInfo()
print("Land cover value at rural point is:", lc_rural_point)

land cover value 12 is:
>Croplands: at least 60% of area is cultivated cropland.

Note the elevation difference also

### Time Series
Using a `getRegion().getInfo()` combo returns a list format.

In [ ]:
# Get the data for the pixel intersecting the point in urban area.
lst_u_poi = lst.getRegion(u_poi, scale).getInfo()

# Get the data for the pixel intersecting the point in rural area.
lst_r_poi = lst.getRegion(r_poi, scale).getInfo()

# Preview the result.
lst_u_poi[:5]

Tutorial points to some None values and that by reading the meatadata, `QC_Day` value of 2 means that cloud interference meant that no calculation of land temperature has happened.

Tutorial then helpfully defines a func to convert to a pd DataFrame:

In [ ]:
import pandas as pd


def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[["longitude", "latitude", "time", *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors="coerce")

    # Convert the time field into a datetime.
    df["datetime"] = pd.to_datetime(df["time"], unit="ms")

    # Keep the columns of interest.
    df = df[["time", "datetime", *list_of_bands]]

    return df

In [ ]:
lst_df_urban = ee_array_to_df(lst_u_poi, ["LST_Day_1km"])
lst_df_urban.head()

In [ ]:
# Nice little temp conversion func too:
def t_modis_to_celsius(t_modis):
    """Converts MODIS LST units to degrees Celsius."""
    t_celsius = 0.02 * t_modis - 273.15
    return t_celsius

In [ ]:
# Apply the function to get temperature in celsius.
lst_df_urban["LST_Day_1km"] = lst_df_urban["LST_Day_1km"].apply(t_modis_to_celsius)
lst_df_urban.head()

In [ ]:
# Do the same for the rural point.
lst_df_rural = ee_array_to_df(lst_r_poi, ["LST_Day_1km"])
lst_df_rural["LST_Day_1km"] = lst_df_rural["LST_Day_1km"].apply(t_modis_to_celsius)

lst_df_rural.head()